In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout,Flatten
from keras import backend as K
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import subprocess

In [3]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
from tensorflow import keras
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.corpus import brown
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [4]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [5]:
sc.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.executor.instances', '3'),
 ('spark.deploy.mode', 'cluster'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.cores', '5'),
 ('spark.executor.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.host', 'kafka1'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.yarn.archive', 'hdfs:///user/spark/conf/spark-libs.jar'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.shuffle.partitions', '200'),
 ('spark.app.id', 'application_1641116526368_0094'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.kryoserializer.buffer.max', '2047'),
 ('spark.executor.memoryOverhead', '1g'),
 ('spark.driver.memoryOverhead', '1g'),
 ('spark.driver.port', '36301'),
 ('spark.executor.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('sp

In [6]:
# def regex_(text):
#     # 영어, 숫자, 특수만문자 제외 삭제.
#     pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
#     text = re.sub(pattern, '', text)
#     pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
#     text = re.sub(pattern, '', text)
#     pattern = '(http|ftp|https):// (?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
#     text = re.sub(pattern, '', text)
#     only_english = re.sub('[^ a-zA-Z]', '', text)
#     only_english = only_english.lower()

#     if bool(only_english and only_english.strip()) and len(only_english) >= 10:
#         return only_english

#     return text

# def get_lemma(word):
#     lemma = wn.morphy(word)
#     if lemma is None:
#         return word
#     else:
#         return lemma


# def get_tokens(sentence):
#     tknzr = TweetTokenizer()
#     tokens = tknzr.tokenize(sentence)
#     tokens = [token for token in tokens if (token not in stop_words and len(token) > 1)]
#     tokens = [get_lemma(token) for token in tokens]
#     return (tokens)

In [7]:
# 새로 들어온 csi
csi_pos_neg = spark.read.csv("hdfs:///user/spark/datafile/csiposneg.csv", header=True)

In [8]:
csi_pos_neg = csi_pos_neg.withColumn("label",col("label").cast("integer"))

In [9]:
# get_tokens 함수과 같은 용도
stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')

In [10]:
r1 = stage_1.transform(csi_pos_neg)
r1.show(5)

+---+--------------------+-----+--------------------+
|_c0|                text|label|               pos_t|
+---+--------------------+-----+--------------------+
|  0|threatmeter hacke...|    1|[threatmeter, hac...|
|  1|first android mal...|    1|[first, android, ...|
|  2|adobe fixes six c...|    1|[adobe, fixes, si...|
|  3| scienceporn  in ...|    1|[scienceporn, in,...|
|  4|riskware hmoyfzb ...|    1|[riskware, hmoyfz...|
+---+--------------------+-----+--------------------+
only showing top 5 rows



In [11]:
# get_lemma 함수와 같은 용도
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

In [12]:
r2 = stage_2.transform(r1)
r2.show(5)

+---+--------------------+-----+--------------------+--------------------+
|_c0|                text|label|               pos_t|      filtered_words|
+---+--------------------+-----+--------------------+--------------------+
|  0|threatmeter hacke...|    1|[threatmeter, hac...|[threatmeter, hac...|
|  1|first android mal...|    1|[first, android, ...|[first, android, ...|
|  2|adobe fixes six c...|    1|[adobe, fixes, si...|[adobe, fixes, si...|
|  3| scienceporn  in ...|    1|[scienceporn, in,...|[scienceporn, vac...|
|  4|riskware hmoyfzb ...|    1|[riskware, hmoyfz...|[riskware, hmoyfz...|
+---+--------------------+-----+--------------------+--------------------+
only showing top 5 rows



In [13]:
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame

class TextToSequence(Transformer):
    
    def __init__(self):
        super(TextToSequence, self).__init__()
    
    def _transform(self, df: DataFrame):
        # 우리가 해야할 일은 dataframe을 순서통일해서 text와 label 리스트로 나누는것
        samples = df.select("text").rdd.flatMap(lambda x: x).collect()
        labels = df.select("label").rdd.flatMap(lambda x: x).collect()
        
        vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
        text_ds = tf.data.Dataset.from_tensor_slices(samples).batch(128)
        vectorizer.adapt(text_ds)
        
        voc = vectorizer.get_vocabulary()
        word_index = dict(zip(voc, range(len(voc))))
        
        x_train = vectorizer(np.array([[s] for s in samples])).numpy()
        zip_array = list(zip(samples, labels, x_train.tolist()))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature'])
        
        #fdf = spark.createDataFrame(df)
        return fdf

In [14]:
text_sequence = TextToSequence()
df_example = text_sequence.transform(r2)

In [15]:
df_example.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                |label|feature                                                                                                    

In [16]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature"]).alias("feature")
)

In [17]:
#df_with_vectors.select('label', 'feature').limit(5).toPandas()

In [18]:
label_str_index = StringIndexer(inputCol='label', outputCol='label_index')
label_df = label_str_index.fit(df_with_vectors).transform(df_with_vectors)

In [19]:
label_df.printSchema()

root
 |-- label: long (nullable = true)
 |-- feature: vector (nullable = true)
 |-- label_index: double (nullable = false)



In [20]:
glom_list =  label_df.rdd.glom().collect()
print(len(glom_list))
summ = 0
for i in glom_list:
    print(len(i))
    summ += len(i)
print(summ)

306
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652
652


In [21]:
nb_classes = label_df.select("label").distinct().count()
nb_classes

2

In [22]:
sc = spark._jsc.sc() 

result1 = sc.getExecutorMemoryStatus().keys() # will print all the executors + driver available

result2 = len([executor.host() for executor in sc.statusTracker().getExecutorInfos() ]) -1

print(result1, end ='\n')
print(result2)

Set(kafka1:42691, kafka1:39993, kafka2:33153, kafka3:34299)
3


In [23]:
import csv

def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ',quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed
glove_path = "/root/spark/glove.6B.100d.txt"
pre_glove = glove2dict(glove_path)

In [24]:
f = open("repo_glove.pkl","rb")
newglove = pickle.load(f)

In [25]:
pre_glove.update(newglove)
len(pre_glove)

400192

In [27]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = pre_glove.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

NameError: name 'word_index' is not defined

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

In [ ]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

In [ ]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("feature")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(model.to_json())
estimator.set_categorical_labels(True) # dense 1이면 False로 설정해야함
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(2)
estimator.set_epochs(10)
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("asynchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

In [ ]:
dl_pipeline = Pipeline(stages=[estimator])

In [ ]:
import time
t1 = time.time()
my_dl = dl_pipeline.fit(label_df)
t2 = time.time() - t1
print(t2)

In [ ]:
# 왜 172.28.0.2는 동작안하는거지? cluster 모드라서? master는 동작안하는거야?

In [ ]:
# 나중에 연결할 코드 실시간 구현시

In [8]:
spark = SparkSession \
        .builder \
        .appName("TwitterStreamingAPI") \
        .config("spark.mongodb.input.uri","mongodb://117.17.189.6:27017/tweet.kafka_tweet")\
        .config("spark.mongodb.output.uri","mongodb://117.17.189.6:27017/tweet.kafka_tweet")\
        .getOrCreate()

In [9]:
spark.sparkContext.setLogLevel("ERROR")

kafka_df = spark.readStream.format("kafka").option("partition.assignment.strategy","range").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "test2").option("startingOffsets", "latest").load()
kafka_df.printSchema()
kafka_df1 = kafka_df.selectExpr("CAST(value AS STRING)", "timestamp")

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [10]:
transaction_detail_schema = StructType() \
        .add("id", StringType()) \
        .add("text", StringType()) \
        .add("created_at", StringType())\

kafka_df2 = kafka_df1 \
        .select(from_json(col("value"), transaction_detail_schema).alias("transaction_detail"), "timestamp")

In [15]:
kafka_df3 = kafka_df2.select("transaction_detail.*", "timestamp")
udf_model = udf(prediction, StringType())

new_df = kafka_df3.withColumn("prediction", udf_model("text"))
# new_df = new_df.drop("text")

In [ ]:
def glove_embedding():
    # exit code 0인지 확인하는 코드 구현 try - except
    subprocess.check_call(["python3", "newGlove.py"]) # should exit with status 0
    
    f = open("repo_glove.pkl","rb")
    newglove = pickle.load(f)
    print(len(newglove))
    num_tokens = len(voc) + 2
    embedding_dim = 100
    hits = 0
    misses = 0

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = pre_glove.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
    print("Converted %d words (%d misses)" % (hits, misses))

    embedding_layer = Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        trainable=False,
    )
    
    int_sequences_input = keras.Input(shape=(None,), dtype="int64")
    embedded_sequences = embedding_layer(int_sequences_input)
    x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(128, 5, activation="relu")(x)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(128, 5, activation="relu")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    preds = layers.Dense(2, activation="softmax")(x)
    model = keras.Model(int_sequences_input, preds)
    model.summary()
    
    # 노드 하나에서만 동작함
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
    )
    model.fit(x_train, y_train, batch_size=128, epochs=1, validation_data=(x_val, y_val))
    
    string_input = keras.Input(shape=(1,), dtype="string")
    x = vectorizer(string_input)
    preds = model(x)
    end_to_end_model = keras.Model(string_input, preds)

#     probabilities = end_to_end_model.predict(
#         [["this message is about computer graphics and 3D modeling"]]
#     )

In [13]:
def prediction(text):
#     max_length = 100
#     pic_str = 'pic.twitter.com/'
#     if text.startswith('RT'):
#         text_l = text.split()
#         text = ' '.join(text_l[2:])
#     text = regex_(text)
#     if isinstance(text, str) == False:
#         return "error"
#     tweet_l = text.split()
#     for t in tweet_l:
#         if pic_str in t:
#             len_text = len(t)
#     idx = text.find(pic_str)
#     if idx != -1:
#         text = text[:idx] + text[idx + len_text:]

#     reg_text = regex_(text)
#     if reg_text == False:
#             return "error"
    
    tmp = list()
    tmp.append(str(text)) # 전처리가 필요할 경우 reg_text
    
    y_prob = model.predict([tmp])
    y_classes = y_prob.argmax(axis=-1)
    prediction = y_classes.tolist()[0]
    return prediction

In [16]:
def write_mongo_row(df, epoch_id):
    mongoURL = "mongodb://117.17.189.6:27017/tweet.kafka_tweet"
    df.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("uri", mongoURL).save()
    df.show()
    pass

In [ ]:
query = new_df.writeStream.trigger(processingTime='1 seconds').foreachBatch(write_mongo_row).start() 
query.awaitTermination()

spark.stop()